# Armor JSON Generator

Generating the requisite resource / data pack JSONs in a Jupyter notebook

In [1]:
import json
from pathlib import Path
from typing import Any, Dict, Optional, Sequence

In [2]:
resources_dir = Path("src") / "main" / "resources"

In [3]:
mod_id: str = "unobtainiumarmor"
armor_pieces: Sequence[str] = ("helmet", "chestplate", "leggings", "boots", "elytra")

## Set the name of the armor material

In [4]:
material: str = "bumblenado"

Also toggle whether we want armored Elytra

In [5]:
armored_elytra: bool = True

In [6]:
armor_pieces = [
    armor_piece
    for armor_piece in armor_pieces
    if armor_piece != "elytra" or armored_elytra is True
]

And set the crafting material ingot

In [7]:
ingot: Optional[str] = "bumblenado_gem"

## Update language files

English only, sorry.

In [8]:
language_file_path = resources_dir / "assets" / mod_id / "lang" / "en_us.json"

In [9]:
with open(language_file_path) as language_file:
    language_lookup: Dict[str, str] = json.load(language_file)

In [10]:
prettify = lambda resource_name: resource_name.replace("_", " ").title()
for armor_piece in armor_pieces:
    item_name = f"{material}_{armor_piece}"
    language_lookup[f"item.{mod_id}.{item_name}"] = prettify(item_name)
if ingot is not None:
    language_lookup[f"item.{mod_id}.{ingot}"] = prettify(ingot)

In [11]:
with open(language_file_path, "w+") as language_file:
    json.dump(language_lookup, language_file, sort_keys=True, indent=2)

## Create Item JSONs

We'll do this by filling out a template

In [12]:
model_items_folder = resources_dir / "assets" / mod_id / "models" / "item"

In [13]:
def create_new_item_json_from_template(item_name: str) -> None:
    """Create a new item JSON file from template

    Parameters
    ----------
    item_name : str
        The name of the item, which must also be the name of the
        item texture file

    Returns
    -------
    None
    """
    json_form: Dict[str, Any] = {
        "parent": "item/generated",
        "textures": {"layer0": f"{mod_id}:item/{item_name}"},
    }
    with open(model_items_folder / f"{item_name}.json", "w+") as item_file:
        json.dump(json_form, item_file, indent=2)

In [14]:
for armor_piece in armor_pieces:
    item_name = f"{material}_{armor_piece}"
    create_new_item_json_from_template(item_name)
if ingot is not None:
    create_new_item_json_from_template(ingot)

## Create Recipes

We'll do this by copying from another material in the mod.

**Note that we're not going to template the recipe for the ingot**

In [15]:
recipes_folder = resources_dir / "data" / mod_id / "recipes" / "armor"

In [16]:
template_material: str = "cats_eye_gem"

In [17]:
def copy_and_adapt_recipe(
    item_name: str, template_item: str, ingredient: str, ingredient_key: str = "m"
) -> None:
    """Create a crafting recipe file for a new item by replacing the ingredient
    in the crafting recipe for another item

    Parameters
    ----------
    item_name : str
        The name of the new item
    template_item : str
        The name of the item whose recipe you're adapting
    ingredient : str or tuple
        The ingredient, optionally including the namespace
        (if no namespace is provided, it will be assumed to be the mod's)
    ingredient_key : str, optional
        The key for the ingredient. If None is provided,
        it's assumed to be "m" (for material)

    Returns
    -------
    None
    """
    with open(recipes_folder / f"{template_item}.json") as template_file:
        recipe = json.load(template_file)
    recipe["key"][ingredient_key] = ingredient
    with open(recipes_folder / f"{item_name}.json", "w+") as recipe_file:
        json.dump(recipe, recipe_file, indent=2)

In [18]:
if ingot is not None:
    for armor_piece in armor_pieces:
        if armor_piece == "elytra":
            continue
        copy_and_adapt_recipe(
            f"{material}_{armor_piece}", f"{template_material}_{armor_piece}", ingot
        )
if armored_elytra is True:
    copy_and_adapt_recipe(
        f"{material}_elytra",
        f"{template_material}_elytra",
        f"{material}_chestplate",
        ingredient_key="c",
    )

## I don't bother with tags

Maybe I should?

## And that's it!

Time to hop on over to the Java side of the house!